In [ ]:
import pandas as pd
from difflib import SequenceMatcher
import re
import pandas as pd
import logging
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium.webdriver.common.action_chains import ActionChains
import os
import re
from urllib.parse import quote
import gc
import csv

## Cleaning data

In [ ]:

path_of_data = "./Data/Data_Scrapping_SE2026_Clean + Ditemukan.csv"
df = pd.read_csv(
    path_of_data,
    delimiter=';',
)

df.head()


In [ ]:
df = df.drop(df.columns[15:27], axis=1)
df.head()

In [ ]:
df.describe

In [ ]:
df["Validasi"].unique()


In [ ]:
# Filter by 'Validasi' column for rows where the value is 'Ditemukan'
df["Validasi"] = (
    df["Validasi"]
    .astype(str)
    .str.strip()
    .str.replace("\\", "", regex=False)
    .str.replace('"', "", regex=False)
)


In [ ]:
df["Validasi"].unique()

In [ ]:
df_filtered = df[df["Validasi"] == "Ditemukan"]

In [ ]:
print(df_filtered.describe)

## Scoring

In [ ]:
def clean_text(text):
    """Membersihkan dan normalisasi teks untuk perbandingan"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # Hapus karakter khusus tapi pertahankan spasi
    text = re.sub(r'[^\w\s]', ' ', text)
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
def calculate_similarity(query, place_name, address):
    """Menghitung similarity score antara query dengan place_name + address"""
    query_clean = clean_text(query)
    place_clean = clean_text(place_name)
    address_clean = clean_text(address)
    
    # Gabungkan place name dan address
    combined = f"{place_clean} {address_clean}"
    
    # Hitung similarity menggunakan SequenceMatcher
    similarity = SequenceMatcher(None, query_clean, combined).ratio()
    
    # Berikan bonus jika place_name sangat mirip dengan query
    place_similarity = SequenceMatcher(None, query_clean, place_clean).ratio()
    
    # Weighted score: 60% dari combined similarity, 40% dari place similarity
    final_score = (similarity * 0.6) + (place_similarity * 0.4)
    
    return final_score


In [ ]:
def process_csv(input_file):
    """Memproses CSV dan melakukan deduplikasi berdasarkan kecocokan query"""
    
    print(f"Melakukan cleaning......................................\n")
    df = pd.DataFrame(input_file)
    def clean_text(x):
        if isinstance(x, str):
            return (
                x.replace('', '')
                .replace('\n', ' ')
                .replace('\r', ' ')
                .strip()
            )
        return x
    df_res = df.map(clean_text)
    # print(df)
    
    # Pastikan kolom yang diperlukan ada
    required_cols = ['idsbr', 'Query', 'Actual Place Name', 'Address']
    missing_cols = [col for col in required_cols if col not in df_res.columns]
    if missing_cols:
        print(f"Error: Kolom yang hilang: {missing_cols}")
        return
    
    print(f"Total baris: {len(df)}")
    
    # Tambahkan kolom untuk similarity score dan validasi
    df_res['similarity_score'] = 0.0
    df_res['Validasi'] = 'Tidak Ditemukan'
    
    # Hitung similarity score untuk setiap baris 
    for idx, row in df_res.iterrows():
        score = calculate_similarity(
            row['Query'],
            row['Actual Place Name'],
            row['Address']
        )
        if score > 0.6:
            df_res.at[idx, 'similarity_score'] = score

    # print(df)
    
    # Group by idsbr untuk mencari winner
    grouped = df_res.groupby('idsbr')
    
    processed_rows = []
    
    for idsbr, group in grouped:
        if pd.isna(idsbr) or str(idsbr).strip() == '':
            # Jika idsbr kosong, tandai sebagai tidak ditemukan
            for idx, row in group.iterrows():
                row_dict = row.to_dict()
                row_dict['Validasi'] = 'Tidak Ditemukan'

                processed_rows.append(row_dict)
            continue
        
        # Urutkan berdasarkan similarity score (descending)
        sorted_group = group.sort_values('similarity_score', ascending=False)
        
        # Ambil winner (score tertinggi)
        winner_idx = sorted_group.index[0]
        
        for idx, row in sorted_group.iterrows():
            row_dict = row.to_dict()
            
            if idx == winner_idx:
                # Ini adalah winner
                row_dict['Validasi'] = 'Ditemukan'
                row_dict['idsbr'] = idsbr
            else:
                # Ini adalah loser, kosongkan idsbr
                row_dict['Validasi'] = 'Tidak Ditemukan'
            
            processed_rows.append(row_dict)


    # Buat dataframe baru dari hasil
    result_df = pd.DataFrame(processed_rows)
    # print(result_df)
    
    # Hapus kolom similarity_score (kolom temporary)
    result_df = result_df.drop('similarity_score', axis=1)
    
    # Urutkan ulang kolom agar Validasi di akhir
    cols = [col for col in result_df.columns if col != 'Validasi']
    cols.append('Validasi')
    result_df = result_df[cols]
    
    kolom_utama = [
                'idsbr', 'Query', 'Actual Place Name', 'Category', 'Rating',
                'Address', 'Phone Number', 'Website', 'Latitude', 'Longitude',
                'Status', 'Open Status', 'Operation Hours'
    ]
                # Ambil kolom utama yang benar-benar ada di DataFrame
    kolom_utama_ada = [c for c in kolom_utama if c in result_df.columns]

                # Ambil kolom tambahan (selain kolom utama)
    kolom_tambahan = [c for c in result_df.columns if c not in kolom_utama_ada]

                # Susun ulang kolom
    result_df = result_df[kolom_utama_ada + kolom_tambahan]
    # Simpan ke file output

    # Tampilkan statistik
    return result_df



In [ ]:
def scorring_data (df_input:pd.DataFrame):
    def clean_text(x):
        if isinstance(x, str):
            return (
                x.replace('', '')
                .replace('\n', ' ')
                .replace('\r', ' ')
                .strip()
            )
        return x
    df_res = df_input.map(clean_text)
    # print(df)
    
    # Pastikan kolom yang diperlukan ada
    required_cols = ['idsbr', 'Query', 'Actual Place Name', 'Address']
    missing_cols = [col for col in required_cols if col not in df_res.columns]
    if missing_cols:
        print(f"Error: Kolom yang hilang: {missing_cols}")
        return
    
    print(f"Total baris: {len(df)}")

    required_cols = ['idsbr', 'Query', 'Actual Place Name', 'Address']
    missing_cols = [col for col in required_cols if col not in df_res.columns]
    if missing_cols:
        print(f"Error: Kolom yang hilang: {missing_cols}")
        return
    
    # Hitung similarity score untuk setiap baris
    for idx, row in df_res.iterrows():
        score = calculate_similarity(
            row['Query'],
            row['Actual Place Name'],
            row['Address']
        )
        df_res.at[idx, 'similarity_score'] = score
    # df_result = df_res

    return df_res

## Clustering by Country 

In [ ]:
# Batas koordinat Surabaya (Update Final 2026)
SURABAYA_BOUNDS = {
    'lat_min': -7.36,  
    'lat_max': -7.15,  
    'lon_min': 112.59, 
    'lon_max': 112.88  
}


In [ ]:

def bersihkan_dan_konversi(nilai, tipe='lat'):
    """Membersihkan format titik ganda dan konversi ke float"""
    try:
        if pd.isna(nilai): return None
        
        # Ubah ke string dan bersihkan karakter non-numerik kecuali minus dan titik
        s = re.sub(r'[^0-9\.\-]', '', str(nilai))
        
        # Jika ada lebih dari satu titik, ambil semua angka dan format ulang
        if s.count('.') > 1:
            digits = s.replace('.', '').replace('-', '')
            sign = "-" if "-" in s else ""
            if tipe == 'lat':
                # Asumsi Latitude Surabaya dimulai dengan -7...
                return float(sign + digits[0] + "." + digits[1:])
            else:
                # Asumsi Longitude Surabaya dimulai dengan 112...
                # Menangani kasus '1.127...' atau '112.7...'
                if digits.startswith('112'):
                    return float(digits[:3] + "." + digits[3:])
                else:
                    return float(digits[:3] + "." + digits[3:])
        
        return float(s)
    except:
        return None


In [ ]:
def cek_lokasi_surabaya(lat_raw, lon_raw):
    """Mengecek apakah koordinat berada di dalam batas Surabaya"""
    lat = bersihkan_dan_konversi(lat_raw, 'lat')
    lon = bersihkan_dan_konversi(lon_raw, 'lon')
    
    if lat is None or lon is None:
        return "error_koordinat"
    
    # Normalisasi otomatis jika angka meledak (misal -72.5 menjadi -7.25)
    if lat < -10: lat = lat / 10
    if lon > 1000: lon = lon / 10
    
    is_lat_in = SURABAYA_BOUNDS['lat_min'] <= lat <= SURABAYA_BOUNDS['lat_max']
    is_lon_in = SURABAYA_BOUNDS['lon_min'] <= lon <= SURABAYA_BOUNDS['lon_max']
    
    if is_lat_in and is_lon_in:
        return "disurabaya"
    else:
        return "tidak disurabaya"

## Start here

In [ ]:
#ambil data yang sudah di bersihkan 
# df = df_filtered

# #scoring process
# df_res = scorring_data(df)
# df_res.head()




In [ ]:
# df_res['similarity_score'].describe()

In [ ]:
# === EKSEKUSI DATA ===
file_path = "./Data/df_res_score.csv"

try:
    # Membaca file dengan pemisah ;
    df = pd.read_csv(file_path, delimiter=',', low_memory=False)

    # # 1. Hapus kolom Column1 sampai Column12 jika ada
    # cols_to_drop = [f'Column{i}' for i in range(1, 13) if f'Column{i}' in df.columns]
    # df = df.drop(columns=cols_to_drop)

    # 2. Tambahkan kolom status_lokasi dengan fungsi yang sudah diperbaiki
    df['status_lokasi'] = df.apply(
        lambda row: cek_lokasi_surabaya(row['Latitude'], row['Longitude']), 
        axis=1
    )

    # 3. Simpan hasil
    output_path = "./DATA/clean_final_sby_all_score.csv"
    df.to_csv(output_path, sep=';', index=False)

    print("=== LAPORAN PROSES 2026 ===")
    print(f"Total Data: {len(df)}")
    print(f"Distribusi:\n{df['status_lokasi'].value_counts()}")
    
    # Tes bukti perbaikan untuk data yang Anda tanyakan
    print("\nTes Data Error (contoh -72.527.579):")
    print(f"Hasil: {cek_lokasi_surabaya('-72.527.579', '112.735.948')}")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

In [ ]:
#grap a data with spesific longtitude 
df = pd.read_csv('./Data/clean_final_sby_all_score.csv',delimiter=';')
display(df)

In [ ]:
df_new = df[df['status_lokasi'] == 'disurabaya' and df['similarity_score'] > 0.6]
